## Setting up AWS Athena


### Step 1: Go to AWS Glue Service to Create and Run a "Crawler"

* A crawler is an automated service in AWS that looks through a set of classifiers prioritized by the schema to create metadata tables.  

* To get the most recent version of the metadata, which is updated by SRA daily, it is necessary to rerun the crawler.

* This differs from BigQuery because BigQuery database and table names are static and the metadata tables are recreated daily by SRA using automated scripts.

<img src="./img/crawler.png" align="left" width="700"/>

### Step 2: Confirm the Name of the Database and the Tables

* When the crawler is created, the user sets the database name. In this workshop, "sra" was the name set by the crawler.

* The level of the tables that show up in the database depends on which datastore is targeted by the crawler. In this case, all the SRA tables are visible because the crawler was set to run on the highest level directory **s3://sra-pub-metadata-us-east-1**

* Note that the tables are all in one database called **"sra"**. This differs from BigQuery which has two sets of tables in its schema: **"sra"** and **"sra_tax_analysis_tool"**


#### Athena Tables

<img src="./img/table.png" align="center" width="700"/>





#### BigQuery Tables

<img src="./img/BigQuery.png" align="center" width="350"/>


The Query workspace for Athena looks very similar to the BigQuery workspace.  The syntax is almost identical, but before starting, make note of how the Database and Tables are structured. The column names are exactly the same between the two providers.

<img src="./img/athena.png" align="center" width="800"/>

### Step 3: Create a "Staging" Bucket

* Athena differs from BigQuery in that it requires a user to create a bucket to hold all the metadata and query outputs

* This is true for both command line interface as well as direct console queries.

### Run the following command to create your bucket for this workshop

This script also applies credentials that allow this notebook (which is on GCP) to access AWS to create buckets and run Athena. It also creates a connection using sqlalchemy to the Athena database.

*The script is unique to this workshop, so make sure to add your own AWS credentials when working outside of this event. Refer to the setup_AWS_credentials.ipynb notebook for tips and tricks on how to do this.*

In [9]:
%run Setup_AWS_Credentials.py

/arhodes4-sql-workshop
BUCKETS	2021-09-15T10:45:40.000Z	arhodes4-sql-workshop
Done.


### Test that the Credentials were Loaded Correctly

In [10]:
%%sql

SELECT acc, bioproject 
FROM sra.metadata
WHERE organism = 'Homo sapiens'
LIMIT 5



Done.


acc,bioproject
ERR817520,PRJEB7886
ERR806508,PRJEB7886
ERR802961,PRJEB7886
ERR824732,PRJEB7886
ERR818574,PRJEB7886


# Challenge : To find RNA-Seq data from estrogen receptor negative and progesterone positive breast cancer cell lines.


Hormone receptor positive (HR+) breast cancer cells may have either estrogen receptors (ER+) or progesterone receptors (PR+) or both.

We have already found one search strategy to find a dataset that had ER+ breast cancer cell lines.

This case study will look for HR+ breast cancer cell lines that are ER- and PR+

This will requre combining search strategies to narrow our choices.

These searchers are taking place on Athena, so the syntax is mostly the same with some subtle differences.

Some of they syntax, such as capitalization or lowercase, is dicated by the tool we are using, so if a query in this notebook does not work exactly the same in Athena, try that first.

### Start with a more general search that uses an "or" to specify receptor types

In [11]:
%%sql


SELECT acc, bioproject, extracted.k, extracted.v
FROM sra.metadata, unnest(attributes) as t(extracted)
WHERE assay_type = 'RNA-Seq'
    AND consent = 'public'
    AND (extracted.v LIKE '%ER-%' or extracted.v LIKE '%HR+%')


Done.


acc,bioproject,k,v
SRR6884902,PRJNA445386,primary_search,ER-S8
SRR6884902,PRJNA445386,primary_search,ER-S8-R2-Q20.fastq
SRR3646855,PRJNA320793,primary_search,SUMMER-CK
SRR6515436,PRJNA431770,primary_search,LIVER-PZQ_1_S47_R1_001.fastq.gz
SRR9094446,PRJNA543872,library_construction_method_sam,SILVER-seq
SRR10355236,PRJNA543872,library_construction_method_sam,SILVER-seq
SRR10355288,PRJNA543872,library_construction_method_sam,SILVER-seq
SRR10355259,PRJNA543872,library_construction_method_sam,SILVER-seq
SRR13166188,PRJNA681542,group_sam,ER-/PR-
SRR13166166,PRJNA681542,group_sam,ER-/PR-


### A lot of the metadata we are picking up is not related to samples. So the next search specifies that the key value needs to end in "sam" by using a wildcard "%" before "sam" and not after.

In [12]:
%%sql


SELECT acc, bioproject, extracted.k, extracted.v
FROM sra.metadata, unnest(attributes) as t(extracted)
WHERE assay_type = 'RNA-Seq'
    AND consent = 'public'
    AND (extracted.v LIKE '%ER-%' or extracted.v LIKE '%HR+%')
    AND extracted.k LIKE '%sam'





Done.


acc,bioproject,k,v
DRR186421,PRJDB8411,phenotype_sam,ER-positive
SRR7764005,PRJNA488315,ecotype_sam,LER-1
ERR1588887,PRJEB15161,ecotype_sam,LER-1
ERR1096835,PRJEB11517,description_sam,"HAP1 FER-KO cells, stimulated with WNT3, replicate R2"
ERR1096835,PRJEB11517,title_sam,"HAP1 FER-KO cells, stimulated with WNT3, replicate R2"
ERR2624527,PRJEB27236,description_sam,"Protocols: ER-alpha positive BC cells: MCF-7 (ATCC HTB-22),were propagated in Dulbecco's modified Eagle medium (DMEM; Sigma-Aldrich) supplemented with 10% FBS (HyClone) and antibiotics: 100 U/ml penicillin, 100 mg/ml streptomycin, 250 ng/ml amphoterici..."
ERR2624528,PRJEB27236,description_sam,"Protocols: ER-alpha positive BC cells: MCF-7 (ATCC HTB-22),were propagated in Dulbecco's modified Eagle medium (DMEM; Sigma-Aldrich) supplemented with 10% FBS (HyClone) and antibiotics: 100 U/ml penicillin, 100 mg/ml streptomycin, 250 ng/ml amphoterici..."
ERR5433883,PRJEB43418,insdc_center_alias_sam,"MARTIN-LUTHER-UNIVERSITY HALLE-WITTENBERG, INSTITU"
ERR5433883,PRJEB43418,insdc_center_name_sam,"MARTIN-LUTHER-UNIVERSITY HALLE-WITTENBERG, INSTITU"
SRR9938248,PRJNA558793,cultivar_sam,KER-6


### Adding the word "receptor" in front of the wildcard here may limit the samples to a reasonable set for the next steps. 

There are several strategies to find interesting data sets, this is one that seems to work just within this case study. A good practice would be to spend some time trying different methods to get the best data set for your research question.

In [13]:
%%sql


SELECT acc, bioproject, extracted.k, extracted.v
FROM sra.metadata, unnest(attributes) as t(extracted)
WHERE assay_type = 'RNA-Seq'
    AND consent = 'public'
    AND (extracted.v LIKE '%ER-%' or extracted.v LIKE '%HR+%')
    AND extracted.k LIKE 'receptor%sam'





Done.


acc,bioproject,k,v
SRR12060826,PRJNA640820,receptor_status_sam,HR+
SRR12060808,PRJNA640820,receptor_status_sam,ER-/PR+
SRR12060807,PRJNA640820,receptor_status_sam,ER-/PR+
SRR12060824,PRJNA640820,receptor_status_sam,HR+
SRR12060827,PRJNA640820,receptor_status_sam,HR+
SRR12060825,PRJNA640820,receptor_status_sam,HR+
SRR12060809,PRJNA640820,receptor_status_sam,ER-/PR+
SRR12060806,PRJNA640820,receptor_status_sam,ER-/PR+


#### This looks like a good set, let's pull out the accession list for downstream analysis.

The syntax is slightly different because we are using sqlmagic - a different program than bigquery.

The output of a query can be saved as a results object which is easily made into a dataframe for downstream analysis.  We are creating a results object called "accessions" and then saving it into a text file called "cell_line_ER_HR.txt"



In [14]:
%%sql accessions <<


SELECT acc
FROM sra.metadata, unnest(attributes) as t(extracted)
WHERE assay_type = 'RNA-Seq'
    AND consent = 'public'
    AND (extracted.v LIKE '%ER-%' or extracted.v LIKE '%HR+%')
    AND extracted.k LIKE 'receptor%sam'

Done.
Returning data to local variable accessions


#### The output is different in SQL magic compared to BigQuery magic in the other notebook.

It is not a dataframe yet, it would need to have accessions.DataFrame() to show the index as in the BigQuery results object.

In [15]:
accessions

acc
SRR12060824
SRR12060825
SRR12060809
SRR12060806
SRR12060827
SRR12060808
SRR12060807
SRR12060826


In [16]:
accessions.DataFrame()

,acc
0,SRR12060824
1,SRR12060825
2,SRR12060809
3,SRR12060806
4,SRR12060827
5,SRR12060808
6,SRR12060807
7,SRR12060826


In [17]:
file = open("cell_line_ER_HR.txt", "w")
file.write(accessions.DataFrame().to_string(index=False, header=False))
file.close()

***

## Example Bioinformatic Workflow

***

### We may not have time to go over this in detail, the code is here for your information ###

<img src="./img/Workflow_demo.png" align="left" width="700"/>




### We are going to skip down to the final analysis for this demo  ###

In [18]:
%%bash

cat cell_line_ER_HR.txt

SRR12060824
SRR12060825
SRR12060809
SRR12060806
SRR12060827
SRR12060808
SRR12060807
SRR12060826

### Configure SRA Toolkit

In [20]:
GUID = !uuidgen
GUID = str(GUID).strip('[]')
!vdb-config --set LIBS/GUID={GUID}
!vdb-config --report-cloud-identity yes
!vdb-config --set /repository/user/main/public/cache-disabled=true
#!cat $HOME/.ncbi/user-settings.mkfg

Report Cloud Instance Identity was set to true


In [22]:
%%bash

cat cell_line_ER_HR.txt | xargs -P 12 -n 1 fastq-dump -X 1000000 -O cell_line_fastq

Read 1000000 spots for SRR12060806
Written 1000000 spots for SRR12060806
Read 1000000 spots for SRR12060808
Written 1000000 spots for SRR12060808
Read 1000000 spots for SRR12060824
Written 1000000 spots for SRR12060824
Read 1000000 spots for SRR12060807
Written 1000000 spots for SRR12060807
Read 1000000 spots for SRR12060826
Written 1000000 spots for SRR12060826
Read 1000000 spots for SRR12060825
Written 1000000 spots for SRR12060825
Read 1000000 spots for SRR12060809
Written 1000000 spots for SRR12060809
Read 1000000 spots for SRR12060827
Written 1000000 spots for SRR12060827


In [23]:
%%bash

ls cell_line_fastq

SRR12060806.fastq
SRR12060807.fastq
SRR12060808.fastq
SRR12060809.fastq
SRR12060824.fastq
SRR12060825.fastq
SRR12060826.fastq
SRR12060827.fastq


In [24]:
%%bash

datasets download genome accession GCF_000001405.39 --chromosomes 17 --include-gtf --exclude-protein --exclude-rna



Downloading: ncbi_dataset.zip    1.98kB 11.3MB/s
Downloading: ncbi_dataset.zip    1.98kB 11.3MB/s
Downloading: ncbi_dataset.zip    1.98kB 11.3MB/s
Downloading: ncbi_dataset.zip    32.8kB 947kB/s
Downloading: ncbi_dataset.zip    32.8kB 947kB/s
Downloading: ncbi_dataset.zip    65.5kB 1.06MB/s
Downloading: ncbi_dataset.zip    65.5kB 1.06MB/s
Downloading: ncbi_dataset.zip    98.3kB 1.19MB/s
Downloading: ncbi_dataset.zip    197kB 2.17MB/s
Downloading: ncbi_dataset.zip    197kB 2.17MB/s
Downloading: ncbi_dataset.zip    229kB 2.08MB/s
Downloading: ncbi_dataset.zip    393kB 3.34MB/s
Downloading: ncbi_dataset.zip    393kB 3.34MB/s
Downloading: ncbi_dataset.zip    426kB 3.09MB/s
Downloading: ncbi_dataset.zip    426kB 3.09MB/s
Downloading: ncbi_dataset.zip    655kB 3.95MB/s
Downloading: ncbi_dataset.zip    1.11MB 6.36MB/s
Downloading: ncbi_dataset.zip    1.15MB 6.51MB/s
Downloading: ncbi_dataset.zip    1.25MB 6.39MB/s
Downloading: ncbi_dataset.zip    1.51MB 7.33MB/s
Downloading: ncbi_dataset.zip 

New version of client (12.12.0) available at https://ftp.ncbi.nlm.nih.gov/pub/datasets/command-line/LATEST/linux-amd64/datasets


In [25]:
%%bash

unzip -o ncbi_dataset.zip -d GRCh38.p13

Archive:  ncbi_dataset.zip
  inflating: GRCh38.p13/README.md    
  inflating: GRCh38.p13/ncbi_dataset/data/assembly_data_report.jsonl  
  inflating: GRCh38.p13/ncbi_dataset/data/GCF_000001405.39/chr17.fna  
  inflating: GRCh38.p13/ncbi_dataset/data/GCF_000001405.39/chr17.unlocalized.scaf.fna  
  inflating: GRCh38.p13/ncbi_dataset/data/GCF_000001405.39/cds_from_genomic.fna  
  inflating: GRCh38.p13/ncbi_dataset/data/GCF_000001405.39/genomic.gff  
  inflating: GRCh38.p13/ncbi_dataset/data/GCF_000001405.39/genomic.gtf  
  inflating: GRCh38.p13/ncbi_dataset/data/GCF_000001405.39/sequence_report.jsonl  
  inflating: GRCh38.p13/ncbi_dataset/data/dataset_catalog.json  


In [26]:
%%bash

makeblastdb -in GRCh38.p13/ncbi_dataset/data/GCF_000001405.39/chr17.fna -dbtype nucl -parse_seqids -out GRCh38.p13/GRCh38.p13





Building a new DB, current time: 09/15/2021 10:52:39
New DB name:   /home/jupyter-arhodes4/ASHG-Workshop-2021/GRCh38.p13/GRCh38.p13
New DB title:  GRCh38.p13/ncbi_dataset/data/GCF_000001405.39/chr17.fna
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 1 sequences in 1.0639 seconds.




In [27]:
%%bash

mkdir cell_line_bams

### Making alignments

This script can be automated, but for the purposes of this demo, we are choosing only two of the fastq files to align to Chromosome 17 - a hot spot of activity for HER genes.  

* SRR12060806 is HR+

* SRR12060824 is ER-/PR+

Doing a little more digging into bioproject PRJNA640820: "62 breast cell lines composed of 27 Triple negative breast cancers (TNBC), 5 Non malignant (NM) lines, 14 Hormone receptor positive (HR+) and 16 Her2 amplified lines (Her2amp) were profiled." This was a relatively recent dataset produced on a NextSeq here in Boston.

<img src="./img/citation.png" align="left" width="700"/>


https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE152908



In [28]:
%%bash

magicblast -query cell_line_fastq/SRR12060806.fastq -db GRCh38.p13/GRCh38.p13 \
-splice T -no_unaligned -infmt fastq -num_threads 8 | \
samtools view -bS | samtools sort -@ 8 -o cell_line_bams/SRR12060806.bam -O BAM

[bam_sort_core] merging from 0 files and 8 in-memory blocks...


In [29]:
%%bash

magicblast -query cell_line_fastq/SRR12060824.fastq -db GRCh38.p13/GRCh38.p13 \
-splice T -no_unaligned -infmt fastq -num_threads 8 | \
samtools view -bS | samtools sort -@ 8 -o cell_line_bams/SRR12060824.bam -O BAM

[bam_sort_core] merging from 0 files and 8 in-memory blocks...


In [30]:
%%bash

samtools index cell_line_bams/SRR12060824.bam
samtools index cell_line_bams/SRR12060806.bam

## Load the Data in the NCBI Genome Data Viewer

These bam files with their indexes need to be loaded somewhere they can be retrieved by the NCBI Genome Data Viewer.

If you want to load your own remote files into GDV Viewer, it is possible using your Github repository for files that are **less than 1 GB**. Just use the following file format based on the location of your file in the repository.


Follow this format:  https:///<i></i>github.com/YOUR-GITHUB-ACCOUNT/YOUR-GITHUB-REPOSITORY/raw/master/PATH-TO-BAM



## See the Two Bams from this Demo

The two aligned bams have been pre-loaded at the following link: 
    

https://www.ncbi.nlm.nih.gov/genome/gdv/browser/genome/?cfg=NCID_1_53497080_130.14.18.128_9146_1631700092_3532671673

which will be good for the next 90 days.






Here are the links to the two bams if you want to load them again, make sure to zoom to Chromosome 17, that is where the reads were aligned. The screenshot below indicates where to enter these URL's to have the tracks show up.


<img src="./img/add_remote_view.png" align="left" width="600"/>


https://github.com/ncbi/ASHG-Workshop-2021/raw/main/bams/SRR12060806.bam


https://github.com/ncbi/ASHG-Workshop-2021/raw/main/bams/SRR12060824.bam

## What did we find???

Actually, starting with a general idea and targeting a specific question but being general in our search terms, this demo shows the power of using SQL to rapidly search millions of records in SRA. 

The speed and specificity of using SQL for will help researchers quickly locate sequences of interest, bring these sequences directly into a workflow and compare the outcomes within the command line interface.



***

#### The GDV link leads to the TP53 gene, an important protein in the development of cancer.



<img src="./img/tp53_gdv.png" align="left" width="600"/>



***

#### The mismatch in the pileups help narrow the scope of where interesting SNPs might be.



<img src="./img/pileup_view.png" align="left" width="600"/>



***

#### One of the cell lines has a SNP that indicates that cisplatin will be a less effective drug treatment.




<img src="./img/drug_response_variant.png" align="left" width="600"/>






***

#### Reading more into the records shows the importance of this variant for many cancer drugs.

<img src="./img/clinvar.png" align="left" width="600"/>

The cell line with the SNP may be a good candidate for drug therapy studies to emulate individuals whose cancers do not respond to cisplatin. 

# Thank you!!!

## Please Take our Survey!

https://nlmenterprise.co1.qualtrics.com/jfe/form/SV_bx6evo2ghfCjkrk
    
    